In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pypsa
import seaborn as sns

In [ ]:
EXPORT = 100
CO2L = 2.00
CLUSTERS = 4
H = 3
LL = "3.0"
DISCOUNT_RATE = 0.13

RUN = "decr_5_full_3H_ws" #"decr_7_3H_ws"
CATEGORY = "postnetworks"
SCENARIO = f"elec_s_{CLUSTERS}_ec_lc{LL}_Co2L{CO2L}_{H}H_2030_{DISCOUNT_RATE}_DF_{EXPORT}export"

#OVERRIDES = PATH + "pypsa-eur-sec/data/override_component_attrs"

PREFIX = "/mnt/c/Users/scl38887/Documents/git/aldehyde/"
OUTPUT = PREFIX + "results"
INPUT = PREFIX + f"workflow/subworkflows/pypsa-earth-sec/results/{RUN}/{CATEGORY}/{SCENARIO}.nc"
OUTPUT_SCENARIO = f"{OUTPUT}/{RUN}/graphics/demand/{SCENARIO}"
OUTPUT_RAW = f"{OUTPUT}/{RUN}/graphics/demand/"

In [ ]:
n = pypsa.Network(INPUT)

Show global constraint and objective value

In [ ]:
n.global_constraints

In [ ]:
n.objective / 1e9 # billion euros

Electrolysis electricity demand, Renewable Energy generation

In [ ]:
# Electrolysis demand
ely_demand = n.links_t.p0.loc[: ,n.links.carrier == "H2 Electrolysis"].sum(axis=1) * n.snapshot_weightings.generators[0] / 1e6
ely_demand_monthly = (ely_demand.groupby(ely_demand.index.month).sum()).round(2) # in TWh
ely_demand_annual = (ely_demand.groupby(ely_demand.index.year).sum()).round(2) # in TWh

In [ ]:
ely_demand_annual

In [ ]:
ely_demand_monthly

In [ ]:
# Get sum of generation of renewable energies
re_carriers = ['onwind', 'solar', 'ror', 'offwind-ac', 'offwind-dc', 'residential rural solar thermal',
       'services rural solar thermal', 'residential urban decentral solar thermal','services urban decentral solar thermal',
       'urban central solar thermal']
re_supply = (n.generators_t.p.loc[:, n.generators.carrier.isin(re_carriers)].sum(axis=1) * n.snapshot_weightings.generators[0] / 1e6)
re_supply_monthly = (re_supply.groupby(re_supply.index.month).sum()).round(2) # in TWh
re_supply_annual = (re_supply.groupby(re_supply.index.year).sum()).round(2) # in TWh

In [ ]:
re_supply_annual

In [ ]:
re_supply_monthly

### Check matching

In [ ]:
(re_supply - ely_demand).min() # Min value must be >= 0 to confirm hourly matching

In [ ]:
(re_supply - ely_demand).plot() # Min value must be >= 0 to confirm hourly matching

In [ ]:
re_supply[:100].plot()  
ely_demand[:100].plot()
# re_supply_monthly.plot()  
# ely_demand_monthly.plot()

In [ ]:
re_supply_monthly - ely_demand_monthly

### Capacity Renewable Energy

In [ ]:
# Get sum of capacity of renewable energies
(n.generators.p_nom_opt[n.generators.carrier.isin(re_carriers)].sum() / 1e3).round(2) # in GW

### Hydro supply

In [ ]:
n.storage_units.carrier.unique()

In [ ]:
(n.storage_units_t.p.loc[:, n.storage_units.carrier == "hydro"].sum(axis=1).sum() * n.snapshot_weightings.stores[0] / 1e6).round(2) # in TWh

In [ ]:
# Get sum of generation of renewable energies
re_carriers = ['offwind']
(n.generators_t.p.loc[:, n.generators.carrier.isin(re_carriers)].sum(axis=1).sum() * n.snapshot_weightings.generators[0] / 1e6).round(2) # in TWh